## **NYC Payroll Anomaly Detection**


## Introduction
Built an unsupervised machine learning system to detect payroll fraud in NYC employee data, specifically targeting salary manipulation and fake overtime claims without requiring labeled training data.

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load data from the NYC Open Data Socrata API
url = "https://data.cityofnewyork.us/resource/k397-673e.json?$limit=50000"
df = pd.read_json(url)

In [3]:
df.head(3)

,fiscal_year,payroll_number,agency_name,last_name,first_name,agency_start_date,work_location_borough,title_description,leave_status_as_of_june_30,base_salary,pay_basis,regular_hours,regular_gross_paid,ot_hours,total_ot_paid,total_other_pay,mid_init
0,2025,67,ADMIN FOR CHILDREN'S SVCS,RAMSAROOP,BRAHASPA,1996-06-23T00:00:00.000,MANHATTAN,ADMINISTRATIVE DIRECTOR OF SOCIAL SERVICES,ACTIVE,139479.0,per Annum,1820.0,131707.45,0.0,0.00,1497.01,NaN
1,2025,67,ADMIN FOR CHILDREN'S SVCS,GABRIEL,CHRISTINE,1996-06-23T00:00:00.000,BRONX,CHILD PROTECTIVE SPECIALIST SUPERVISOR,ACTIVE,100635.0,per Annum,1820.0,97503.18,0.0,0.00,9120.44,M
2,2025,67,ADMIN FOR CHILDREN'S SVCS,FREDERIQUE,JOYCE,1996-06-23T00:00:00.000,BROOKLYN,CHILD WELFARE SPECIALIST SUPERVISOR,ACTIVE,102880.0,per Annum,1820.0,99678.63,239.5,14825.04,5054.29,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   fiscal_year                 50000 non-null  int64  
 1   payroll_number              50000 non-null  int64  
 2   agency_name                 50000 non-null  object 
 3   last_name                   50000 non-null  object 
 4   first_name                  50000 non-null  object 
 5   agency_start_date           50000 non-null  object 
 6   work_location_borough       50000 non-null  object 
 7   title_description           50000 non-null  object 
 8   leave_status_as_of_june_30  50000 non-null  object 
 9   base_salary                 50000 non-null  float64
 10  pay_basis                   50000 non-null  object 
 11  regular_hours               50000 non-null  float64
 12  regular_gross_paid          50000 non-null  float64
 13  ot_hours                    500

In [5]:
print(df.columns)

Index(['fiscal_year', 'payroll_number', 'agency_name', 'last_name',
       'first_name', 'agency_start_date', 'work_location_borough',
       'title_description', 'leave_status_as_of_june_30', 'base_salary',
       'pay_basis', 'regular_hours', 'regular_gross_paid', 'ot_hours',
       'total_ot_paid', 'total_other_pay', 'mid_init'],
      dtype='object')


In [6]:
df.describe()

,fiscal_year,payroll_number,base_salary,regular_hours,regular_gross_paid,ot_hours,total_ot_paid,total_other_pay
count,50000.000000,50000.0,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,2022.522820,67.0,72607.819446,1451.237142,59189.748297,109.539023,5285.210818,3959.973488
std,1.691513,0.0,26603.575796,639.321193,35776.817944,211.147790,11216.747980,6034.672972
min,2020.000000,67.0,15.500000,-518.000000,-20547.520000,-68.000000,-5178.100000,-56667.130000
25%,2021.000000,67.0,58782.000000,1226.125000,37515.800000,0.000000,0.000000,249.282500
50%,2023.000000,67.0,65921.000000,1820.000000,60161.660000,7.000000,302.580000,2326.655000
75%,2024.000000,67.0,85441.500000,1820.000000,79880.350000,132.000000,5647.817500,5113.935000
max,2025.000000,67.0,286627.000000,2240.000000,332518.520000,2420.000000,239066.510000,158522.440000


In [7]:
## Checking if base salary has any negative values
print((df['base_salary'] <= 0).sum())

0


In [8]:
def engineer_features(df):
  ##Create features for detecting salary and overtime anomalies
  features = df.copy()

  if 'title_description' in df.columns:
    title_avg = df.groupby('title_description')['base_salary'].transform('mean')
    features['salary_vs_title_avg'] = (df['base_salary'] - title_avg) / title_avg

  if 'agency_name' in df.columns:
    agency_avg = df.groupby('agency_name')['base_salary'].transform('mean')
    features['salary_vs_agency_avg'] = (df['base_salary'] - agency_avg) / agency_avg

  if 'total_ot_paid' in df.columns and 'base_salary' in df.columns:
    features['ot_ratio'] = df['total_ot_paid'] / (df['base_salary'] + 1)

  if 'ot_hours' in df.columns:
    features['ot_hours_ratio'] = df['ot_hours'] / 2080

  if 'regular_gross_paid' in df.columns and 'base_salary' in df.columns:
    features['pay_discrepancy'] = abs(df['regular_gross_paid'] - df['base_salary'])

  return features

In [9]:
def prepare_features_for_model(df):
    """
    Prepare numeric and categorical features for Isolation Forest
    Uses one-hot encoding for categorical variables
    """
    numeric_features = [col for col in ['base_salary', 'regular_gross_paid', 'ot_hours', 'total_ot_paid',
                                         'salary_vs_title_avg', 'salary_vs_agency_avg', 'ot_ratio']
                        if col in df.columns]

    categorical_features = [col for col in ['agency_name', 'title_description', 'work_location_borough']
                           if col in df.columns]

    preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),

        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
        ]), categorical_features)
    ]
)

    feature_cols = numeric_features + categorical_features
    X = df[feature_cols].copy()
    X_transformed = preprocessor.fit_transform(X)

    return X_transformed, preprocessor, numeric_features, categorical_features

In [10]:
engineer_features(df)

,fiscal_year,payroll_number,agency_name,last_name,first_name,agency_start_date,work_location_borough,title_description,leave_status_as_of_june_30,base_salary,...,regular_gross_paid,ot_hours,total_ot_paid,total_other_pay,mid_init,salary_vs_title_avg,salary_vs_agency_avg,ot_ratio,ot_hours_ratio,pay_discrepancy
0,2025,67,ADMIN FOR CHILDREN'S SVCS,RAMSAROOP,BRAHASPA,1996-06-23T00:00:00.000,MANHATTAN,ADMINISTRATIVE DIRECTOR OF SOCIAL SERVICES,ACTIVE,139479.0,...,131707.45,0.0,0.00,1497.01,NaN,0.197821,0.920991,0.000000,0.000000,7771.55
1,2025,67,ADMIN FOR CHILDREN'S SVCS,GABRIEL,CHRISTINE,1996-06-23T00:00:00.000,BRONX,CHILD PROTECTIVE SPECIALIST SUPERVISOR,ACTIVE,100635.0,...,97503.18,0.0,0.00,9120.44,M,0.153598,0.386008,0.000000,0.000000,3131.82
2,2025,67,ADMIN FOR CHILDREN'S SVCS,FREDERIQUE,JOYCE,1996-06-23T00:00:00.000,BROOKLYN,CHILD WELFARE SPECIALIST SUPERVISOR,ACTIVE,102880.0,...,99678.63,239.5,14825.04,5054.29,NaN,0.234071,0.416927,0.144099,0.115144,3201.37
3,2025,67,ADMIN FOR CHILDREN'S SVCS,GIBSON,VERNON,1996-06-23T00:00:00.000,MANHATTAN,ASSOCIATE STAFF ANALYST,CEASED,88434.0,...,-1324.76,0.0,0.00,0.00,G,0.004032,0.217968,0.000000,0.000000,89758.76
4,2025,67,ADMIN FOR CHILDREN'S SVCS,BLENMAN,ROBERT,1996-06-23T00:00:00.000,BROOKLYN,CHILD PROTECTIVE SPECIALIST SUPERVISOR,ACTIVE,78902.0,...,76441.54,0.0,0.00,7582.82,D,-0.095532,0.086687,0.000000,0.000000,2460.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2020,67,ADMIN FOR CHILDREN'S SVCS,YNOA,CRYSTAL,2019-07-08T00:00:00.000,MANHATTAN,SUMMER COLLEGE INTERN,CEASED,15.5,...,4553.13,0.0,0.00,108.50,NaN,-0.172009,-0.999787,0.000000,0.000000,4537.63
49996,2020,67,ADMIN FOR CHILDREN'S SVCS,DUKE,JAMAL,2017-11-13T00:00:00.000,MANHATTAN,CHILD PROTECTIVE SPECIALIST,CEASED,57070.0,...,4587.65,0.0,0.00,66.16,N,-0.077949,-0.213997,0.000000,0.000000,52482.35
49997,2020,67,ADMIN FOR CHILDREN'S SVCS,SIEGEL,KENNETH,2003-08-25T00:00:00.000,MANHATTAN,AGENCY ATTORNEY,CEASED,79552.0,...,3835.06,0.0,0.00,812.96,I,-0.180691,0.095640,0.000000,0.000000,75716.94
49998,2020,67,ADMIN FOR CHILDREN'S SVCS,COLEMAN,ASABERRY,2019-09-09T00:00:00.000,BROOKLYN,SPECIAL OFFICER,CEASED,33819.0,...,4499.37,0.0,0.00,102.06,R,-0.139573,-0.534224,0.000000,0.000000,29319.63


In [11]:
def detect_salary_anomalies(df, contamination=0.05):

  salary_features = [col for col in ['base_salary', 'regular_gross_paid', 'salary_vs_title_avg', 'salary_vs_agency_avg', 'pay_discrepancy']
                     if col in df.columns]

  X = df[salary_features].fillna(0).values

  scaler = StandardScaler()
  X_scaled = scaler.fit_transform(X)

  ##Isoltaion forest model
  model = IsolationForest(
      contamination=contamination,
      random_state=42,
      n_estimators=100
  )

  predictions = model.fit_predict(X_scaled)
  scores = model.score_samples(X_scaled)

  return predictions, scores, model, scaler

In [12]:
def detect_overtime_anomalies(df, contamination=0.05):

  ot_features = [col for col in ['ot_hours', 'total_ot_paid', 'ot_ratio', 'ot_hours_ratio']
                 if col in df.columns]

  X = df[ot_features].fillna(0).values

  scaler = StandardScaler()
  X_scaled = scaler.fit_transform(X)

  model = IsolationForest(
        contamination=contamination,
        random_state=42,
        n_estimators=100
    )

  predictions = model.fit_predict(X_scaled)
  scores = model.score_samples(X_scaled)

  return predictions, scores, model, scaler

In [13]:
def detect_combined_anomalies(df, contamination=0.05):

  X_transformed, preprocessor, numerical_features, categorical_features = prepare_features_for_model(df)

  model = IsolationForest(
        contamination=contamination,
        random_state=42,
        n_estimators=100,
        max_samples='auto'
    )

  predictions = model.fit_predict(X_transformed)
  scores = model.score_samples(X_transformed)

  return predictions, scores, model, preprocessor

In [14]:
def detect_concept_drift(old_df, new_df, features=['base_salary', 'ot_hours']):
    """
    Monitor for concept drift between old and new data
    """
    drift_detected = False
    drift_details = []

    for feature in features:
        if feature in old_df.columns and feature in new_df.columns:
            old_mean = old_df[feature].mean()
            new_mean = new_df[feature].mean()
            old_std = old_df[feature].std()
            new_std = new_df[feature].std()

            mean_shift = abs(new_mean - old_mean) / (old_mean + 0.001)
            std_shift = abs(new_std - old_std) / (old_std + 0.001)

            if mean_shift > 0.15 or std_shift > 0.25:
                drift_detected = True
                drift_details.append({
                    'feature': feature,
                    'mean_shift': mean_shift,
                    'std_shift': std_shift
                })

    return drift_detected, drift_details

In [15]:
def process_realtime_record(record, salary_model, ot_model, salary_scaler, ot_scaler, reference_df):
    """
    Process a single new record in real-time
    """
    if 'title_description' in record and 'title_description' in reference_df.columns:
        title_avg = reference_df[reference_df['title_description'] == record['title_description']]['base_salary'].mean()
        salary_vs_title = (record['base_salary'] - title_avg) / (title_avg + 1)
    else:
        salary_vs_title = 0

    if 'agency_name' in record and 'agency_name' in reference_df.columns:
        agency_avg = reference_df[reference_df['agency_name'] == record['agency_name']]['base_salary'].mean()
        salary_vs_agency = (record['base_salary'] - agency_avg) / (agency_avg + 1)
    else:
        salary_vs_agency = 0

    salary_features = np.array([[
        record.get('base_salary', 0),
        record.get('regular_gross_paid', 0),
        salary_vs_title,
        salary_vs_agency,
        abs(record.get('regular_gross_paid', 0) - record.get('base_salary', 0))
    ]])

    salary_scaled = salary_scaler.transform(salary_features)
    salary_pred = salary_model.predict(salary_scaled)[0]
    salary_score = salary_model.score_samples(salary_scaled)[0]

    ot_ratio = record.get('total_ot_paid', 0) / (record.get('base_salary', 1) + 1)
    ot_hours_ratio = record.get('ot_hours', 0) / 2080

    ot_features = np.array([[
        record.get('ot_hours', 0),
        record.get('total_ot_paid', 0),
        ot_ratio,
        ot_hours_ratio
    ]])

    ot_scaled = ot_scaler.transform(ot_features)
    ot_pred = ot_model.predict(ot_scaled)[0]
    ot_score = ot_model.score_samples(ot_scaled)[0]

    return {
        'salary_anomaly': salary_pred == -1,
        'salary_score': salary_score,
        'overtime_anomaly': ot_pred == -1,
        'overtime_score': ot_score
    }

## Visualizations

In [16]:
def visualize_salary_anomalies(df):
    """
    Create scatter plot showing salary anomalies by job title
    """
    df['anomaly_label'] = df['salary_anomaly'].replace({True: 'Anomaly', False: 'Normal'})

    fig = px.scatter(
        df,
        x='title_description',
        y='base_salary',
        color='anomaly_label',
        title='NYC Base Salary Anomalies by Job Title',
        labels={'base_salary': 'Base Salary', 'title_description': 'Job Title'},
        opacity=0.7
    )

    fig.update_layout(
        xaxis_tickangle=45,
        template='plotly_white',
        legend_title='Type'
    )

    fig.show()

In [17]:
def visualize_salary_distribution(df):
    """
    Create histogram showing salary distribution with anomalies highlighted
    """
    normal_salaries = df[~df['salary_anomaly']]['base_salary']
    anomaly_salaries = df[df['salary_anomaly']]['base_salary']

    fig = go.Figure()

    fig.add_trace(go.Histogram(
        x=normal_salaries,
        name='All Salaries',
        marker_color='lightblue',
        opacity=0.7
    ))

    fig.add_trace(go.Histogram(
        x=anomaly_salaries,
        name='Anomalies',
        marker_color='red',
        opacity=0.7
    ))

    fig.update_layout(
        title='NYC Base Salary Anomalies',
        xaxis_title='Base Salary',
        yaxis_title='Frequency',
        barmode='overlay',
        template='plotly_white'
    )

    fig.show()

In [18]:
def visualize_overtime_anomalies(df):
    """
    Create scatter plot for overtime anomalies
    """
    df['ot_anomaly_label'] = df['overtime_anomaly'].replace({True: 'Anomaly', False: 'Normal'})

    fig = px.scatter(
        df,
        x='ot_hours',
        y='total_ot_paid',
        color='ot_anomaly_label',
        title='Overtime Hours vs Pay - Anomaly Detection',
        labels={'ot_hours': 'Overtime Hours', 'total_ot_paid': 'Overtime Pay'},
        opacity=0.6
    )

    fig.update_layout(
        template='plotly_white',
        legend_title='Type'
    )

    fig.show()

## Main execution

In [19]:
def main():
  # Batch processing
  processed_df = engineer_features(df)

  sal_pred, sal_scores, sal_model, sal_scaler = detect_salary_anomalies(processed_df, contamination=0.05)
  processed_df['salary_anomaly'] = sal_pred == -1
  processed_df['salary_score'] = sal_scores

  ot_pred, ot_scores, ot_model, ot_scaler = detect_overtime_anomalies(processed_df, contamination=0.05)
  processed_df['overtime_anomaly'] = ot_pred == -1
  processed_df['overtime_score'] = ot_scores

  combined_pred, combined_scores, combined_model, preprocessor = detect_combined_anomalies(processed_df, contamination=0.05)
  processed_df['combined_anomaly'] = combined_pred == -1
  processed_df['combined_score'] = combined_scores

  total = len(processed_df)
  salary_anomalies = processed_df['salary_anomaly'].sum()
  ot_anomalies = processed_df['overtime_anomaly'].sum()
  combined_anomalies = processed_df['combined_anomaly'].sum()

  print(f"Total records analyzed: {total}")
  print(f"Salary anomalies detected: {salary_anomalies}")
  print(f"Overtime anomalies detected: {ot_anomalies}")
  print(f"Combined anomalies detected: {combined_anomalies}")

  if salary_anomalies > 0:
    top_salary = processed_df[processed_df['salary_anomaly']].nsmallest(5, 'salary_score')
    print("\nTop salary anomalies:")
    for idx, row in top_salary.iterrows():
        print(f"{row.get('last_name', 'N/A')}, {row.get('first_name', 'N/A')} - {row.get('title_description', 'N/A')} - ${row.get('base_salary', 0):,.2f}")

  if ot_anomalies > 0:
    top_ot = processed_df[processed_df['overtime_anomaly']].nsmallest(5, 'overtime_score')
    print("\nTop overtime anomalies:")
    for idx, row in top_ot.iterrows():
        print(f"{row.get('last_name', 'N/A')}, {row.get('first_name', 'N/A')} - OT Hours: {row.get('ot_hours', 0):,.1f}")

  # Concept drift detection
  print("\nConcept Drift Detection:")
  new_url = "https://data.cityofnewyork.us/resource/k397-673e.json?$limit=10000&$offset=50000"
  new_df = pd.read_json(new_url)
  new_df = engineer_features(new_df)

  drift_detected, drift_details = detect_concept_drift(processed_df, new_df)

  if drift_detected:
    print("Drift detected in features:")
    for detail in drift_details:
      print(f"{detail['feature']}: mean shift = {detail['mean_shift']:.2%}, std shift = {detail['std_shift']:.2%}")
    print("Recommendation: Retrain models with new data")
  else:
    print("No significant drift detected")

  # Real-time processing
  print("\nReal-time Detection:")
  test_record = {
      'last_name': 'TEST',
      'first_name': 'EMPLOYEE',
      'title_description': 'CHILD PROTECTIVE SPECIALIST',
      'agency_name': 'ADMIN FOR CHILDREN\'S SVCS',
      'base_salary': 150000,
      'regular_gross_paid': 150000,
      'ot_hours': 1200,
      'total_ot_paid': 65000
  }

  rt_result = process_realtime_record(test_record, sal_model, ot_model, sal_scaler, ot_scaler, processed_df)

  if rt_result['salary_anomaly']:
    print(f"Salary anomaly detected in real-time record")
  if rt_result['overtime_anomaly']:
    print(f"Overtime anomaly detected in real-time record")

  # Visualizations
  print("\nGenerating visualizations...")
  visualize_salary_anomalies(processed_df)
  visualize_salary_distribution(processed_df)
  visualize_overtime_anomalies(processed_df)


In [20]:
if __name__ == "__main__":
    main()

Output hidden; open in https://colab.research.google.com to view.

## Project Summary

### Tasks Completed
1. Data Loading & Exploration

Loaded 50,000 records from NYC Open Data API
Explored dataset structure with 17 columns including base_salary, ot_hours, title_description
Verified data quality and checked for missing values

2. Feature Engineering

Created comparison features: salary vs title average, salary vs agency average
Built overtime ratios: OT pay/base salary, OT hours/full-time hours
Calculated pay discrepancies between base and actual gross paid

3. Anomaly Detection (Unsupervised Learning)

Algorithm: Isolation Forest (contamination=0.05)
Salary Detection: Flags unusual compensation patterns by job title and agency
Overtime Detection: Identifies excessive overtime hours and pay
Combined Detection: Uses all features with one-hot encoding for categorical variables

4. Concept Drift Handling

Loads new data batch (10,000 records with offset)
Compares mean and standard deviation shifts
Triggers retraining alert if >15% mean shift or >25% std shift detected

5. Pipeline Implementation

Batch Pipeline: Processes entire dataset at once, generates top anomalies
Real-Time Pipeline: process_realtime_record() function evaluates single transactions instantly

6. Visualization

Scatter plot showing salary anomalies by job title
Histogram comparing normal vs anomalous salary distribution
Overtime hours vs pay scatter plot with anomaly highlighting

### Algorithm Selection Rationale

Isolation Forest chosen because:
- Works without labeled data (unsupervised)
- Efficient for high-dimensional data
- Isolates anomalies based on how easily they separate from normal patterns
- Handles both numeric and categorical features after preprocessing

### Evaluation Strategy (No Labels):
- Count anomalies detected (5% contamination rate)
- Manual review of top 5 suspicious cases for salary and overtime
- Compare results across three detection approaches (salary-only, overtime-only, combined)
- Visual inspection through interactive plots

### Deployment Plan
- Schedule daily batch processing for all payroll records
- Deploy real-time API endpoint for new transaction validation
- Set up weekly drift monitoring to check model validity
- Retrain models monthly or when drift threshold exceeded
- Human review queue for flagged anomalies above threshold

### Conclusion
Successfully implemented a complete anomaly detection system meeting all requirements: uses unsupervised learning (Isolation Forest), detects both salary and overtime fraud, handles concept drift monitoring, and provides both batch and real-time processing capabilities. The system identified anomalies without requiring labeled training data and can adapt to changing payroll patterns over time.

### References:

I had collected data and explored websites for the project. Highlighting the references below:

https://community.sap.com/t5/human-capital-management-blog-posts-by-sap/ai-based-anomaly-monitoring-using-sap-payroll-control-center/ba-p/13965841

https://www.next.gr/ai/ai-in-finance/payroll-anomaly-detection-with-ml

https://www.neeyamo.com/blog/impact-machine-learning-payroll-fraud-detection

#### Dataset link

https://data.cityofnewyork.us/City-Government/Citywide-Payroll-Data-Fiscal-Year-/k397-673e/about_data